In [11]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load data from CSV files
btc_data_daily = pd.read_csv('btc_data_daily.csv', index_col='time', parse_dates=True)

# Filter numeric columns only
numeric_cols = ['close', 'high', 'low', 'open', 'volumefrom', 'volumeto']
btc_data_daily = btc_data_daily[numeric_cols]

# Handle missing values by forward filling
btc_data_daily.fillna(method='ffill', inplace=True)

# Normalize the data
scaler = MinMaxScaler()
btc_data_daily_scaled = scaler.fit_transform(btc_data_daily)
btc_data_daily_scaled = pd.DataFrame(btc_data_daily_scaled, columns=btc_data_daily.columns, index=btc_data_daily.index)

# Define the LSTM model with more layers and units
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(2, 1, self.hidden_layer_size),
                            torch.zeros(2, 1, self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq, self.hidden_cell)
        predictions = self.linear(lstm_out[:, -1])
        return predictions

# Prepare data for training
def prepare_data(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps, 0])  # Extract only the closing price for the target
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32).unsqueeze(1)

# Example with daily data
time_steps = 60  # Use last 60 days for prediction
X_daily, y_daily = prepare_data(btc_data_daily_scaled.values, time_steps)

# Create the model
input_size = X_daily.shape[2]
hidden_layer_size = 100  # Increase hidden units
output_size = 1
model = LSTMModel(input_size, hidden_layer_size, output_size)

# Define loss and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # Lower learning rate

# Train the model
epochs = 100  # Increase the number of epochs
for epoch in range(epochs):
    model.train()
    total_loss = 0
    total_mape = 0
    for i in range(len(X_daily)):
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(2, 1, model.hidden_layer_size),
                             torch.zeros(2, 1, model.hidden_layer_size))

        y_pred = model(X_daily[i].unsqueeze(0))
        y_target = y_daily[i].view_as(y_pred)  # Match the shape of y_pred
        single_loss = loss_function(y_pred, y_target)
        single_loss.backward()
        optimizer.step()

        total_loss += single_loss.item()
        total_mape += (torch.mean(torch.abs((y_target - y_pred) / y_target)) * 100).item()

    avg_loss = total_loss / len(X_daily)
    avg_mape = total_mape / len(X_daily)
    print(f'Epoch {epoch+1} Loss: {avg_loss:.4f}, MAPE: {avg_mape:.4f}%')

# Save the model
torch.save(model.state_dict(), 'btc_trading_model.pth')


Epoch 1 Loss: 0.0017, MAPE: 13.0880%
Epoch 2 Loss: 0.0030, MAPE: 23.7373%
